# Content
1. [SimpleRenderer](#SimpleRenderer)
2. [Sizing](#Sizing)
3. [RemoteRenderer](#RemoteRenderer)
    1. [Default Behaviour](#Default-Behaviour)
    2. [Reset Camera](#Reset-Camera)
    3. [Fixed views](#Fixed-views)
    4. [Using Jupyter Server Proxy](#Using-Jupyter-Server-Proxy)
    5. [Using a custom protocol](#Using-a-custom-protocol)
    6. [Using the webserver with a pvserver](#Using-the-webserver-with-a-pvserver)

# SimpleRenderer

Does nothing more than show the ParaViewWeb server application in the Output Area.

Start a ParaViewWeb server application over the command line (see the [official ParaViewWeb example](https://kitware.github.io/paraviewweb/examples/RemoteRenderer.html#Using-ParaView-as-server))
```bash
pvpython pv_server.py --port 1234 --authKey wslink-secret 
```

In [ ]:
from pvlink import SimpleRenderer

simple = SimpleRenderer(sessionURL='ws://localhost:1234/ws', authKey='wslink-secret')
display(simple)

# Sizing
Widgets scale with container size.

In [ ]:
from ipywidgets import Box

Box(children=[simple], layout={'height':'800px'})

# RemoteRenderer

This renderer requires the `paraview.simple` and `paraview.web` modules.

## Default Behaviour

Upon initialization, the RemoteRenderer starts a webserver for you. If nothing is specified, the webserver will try to start on port 8080 or the next free port thereafter and create a random authentication key. You can pass your own arguments to the webserver. To display help on the possible arguments, you can call `RemoteRenderer.webserver_arguments_help()`.

Arguments that do not take a value need to be set to True.  

**Example:** `debug=True` corresponds to `--debug`

In [1]:
from pvlink import RemoteRenderer

print(RemoteRenderer.webserver_arguments_help())

ParaView Web Server

optional arguments:
  -h, --help            show this help message and exit
  -d, --debug           log debugging messages to stdout
  -s, --nosignalhandlers
                        Prevent Twisted to install the signal handlers so it
                        can be started inside a thread.
  -i HOST, --host HOST  the interface for the web-server to listen on
                        (default: localhost)
  -p PORT, --port PORT  port number for the web-server to listen on (default:
                        8080)
  -t TIMEOUT, --timeout TIMEOUT
                        timeout for reaping process on idle in seconds
                        (default: 300s)
  -c CONTENT, --content CONTENT
                        root for web-pages to serve (default: none)
  -a AUTHKEY, --authKey AUTHKEY
                        Authentication key for clients to connect to the
                        WebSocket.
  -f, --force-flush     If provided, this option will force additional padding
   

In [ ]:
from pvlink import RemoteRenderer

active_renderer = RemoteRenderer(port=8080)
display(active_renderer)

At this point, there is no view or sources, so the output will be a blank canvas.

In [ ]:
from paraview import simple
from pvlink.utility import SetRecommendedRenderSettings

# Create a view and...
view1 = simple.CreateView('RenderView', 'view1')
# ...disable interactor-based render calls and
# ensure pvserver-side rendering (if applicable)
SetRecommendedRenderSettings(view1)
# Create and show a source
source = simple.Cone()
simple.Show(source, view1)
# Update the renderer widget to display the changes
active_renderer.update_render()

## Reset Camera

Function to reset the camera and set the center point to be in the middle of the visible view. Chaning the focal point to the middle of the new view is important, otherwise rotating the view will be done around the old center point.

In [ ]:
from pvlink.utility import ResetCamera
ResetCamera(view1, active_renderer)

## Fixed views

By default, the active view (`viewID = "-1"`) will be shown in the render widget. To set the output of the widget to a fixed view, we can specify the viewID as string.

In [ ]:
view1_ID = view1.GetGlobalIDAsString()
print(view1_ID)

In [ ]:
view1_renderer = RemoteRenderer(port=8083, viewID=view1_ID)
display(view1_renderer)

If we now create a second view, the view will only change in `active_renderer`.

In [ ]:
view2 = simple.CreateView('RenderView', 'view2')
view2.Background = [0,0,0]
SetRecommendedRenderSettings(view2)
active_renderer.update_render()
display(active_renderer)

## Using Jupyter Server Proxy

If you want to access your webserver using Jupyter Server Proxy, you need to set `useJupyterServerProxy` to True and specify the baseURL.

**Example:** If your notebook url is `http://localhost:8888` and you would access a process using `http://localhost:88888/proxy/8080`, the baseURL would be the part before "proxy", `localhost:8888`.

If your notebook is running on HTTPS, set `useJupyterServerProxyHttps` to True to use the HTTPS certificates of the notebook server to keep the connection between the client browser and the Jupyter Server Proxy encrypted.

In both cases, `disableExternalPort` needs to be set to disable port checking during the WebSocket opening handshake. Otherwise, the connection will fail with a port mismatch error since the notebook and the webserver do not run on the same port.

In [ ]:
proxied_renderer = RemoteRenderer(baseURL='localhost:8888', 
                                  disableExternalPort=True,
                                  useJupyterServerProxy=True, # or useJupyterServerProxyHttps
                                  port=8088)

In [ ]:
view1.GetGlobalIDAsString()

In [ ]:
proxied_renderer.viewID = view1.GetGlobalIDAsString()
display(proxied_renderer)

## Using a custom protocol
You can define your own pipeline which should be run when the webserver starts up.

First, define a class which inherits from `pv_wslink.PVServerProtocol` and add your pipeline there. Per default, the RemoteRenderer shows the active view.

To ensure that the view you created in the pipeline is the one shown in the rendering widget and does not get replaced by a new active view, we need to make it available to the outside and later on bind the viewID to the rendering widget. 

In [ ]:
from paraview import simple
from paraview.web import pv_wslink
from paraview.web import protocols as pv_protocols

from pvlink.utility import SetRecommendedRenderSettings


class DemoServer(pv_wslink.PVServerProtocol):
    authKey = 'wslink-secret'

    def initialize(self):
        # Bring used components
        self.registerVtkWebProtocol(pv_protocols.ParaViewWebMouseHandler())
        self.registerVtkWebProtocol(pv_protocols.ParaViewWebViewPort())
        self.registerVtkWebProtocol(pv_protocols.ParaViewWebViewPortImageDelivery())
        # Update authentication key to use
        self.updateSecret(DemoServer.authKey)

        # Your pipeline
        demo_view = simple.CreateView('RenderView', 'SphereView')
        SetRecommendedRenderSettings(demo_view)
        demo_viewID = demo_view.GetGlobalIDAsString()
        # Make the viewID available so we can bind our widget to the correct view
        self.setSharedObject('viewID', demo_viewID)
        sphere = simple.Sphere()
        simple.Show(sphere, demo_view)

In [ ]:
from pvlink import RemoteRenderer

custom_renderer = RemoteRenderer(protocol=DemoServer, a='mysecretkey')
# Bind the viewID to the widget to avoid it showing a different view
# when a different view is set to the active view
custom_renderer.viewID = custom_renderer.protocol.getSharedObject('viewID')
display(custom_renderer)

Since we did not bind a view to `renderer`, the sphere should now also be visible in the `renderer` [output](#Default-Behaviour) after an update call.

In [ ]:
active_renderer.update_render()

## Using the webserver with a pvserver

If `pvserverHost` and `pvserverPort` (default 11111) are specified, the webserver will try to establish a connection to a ParaView server (pvserver) at the given host and port. The pvserver can then take over the heavy lifting and handle very large geometries. Locally, only the image data of the processed data is recieved. To prevent data from being rendered locally, we recommend using the `SetRecommendedRenderSettings` function from `pvlink.utilities` on your displayed view.

If using a custom protocol, the connection has to be created in the protocol or later on manually in the notebook by the user.

In [ ]:
renderer = RemoteRenderer(pvserverHost="localhost", pvserverPort=11111, port=1234)